In [ ]:
import sys 
sys.path.append('..')
# sys.path.insert(0,'/home/ildefog/pydyno')
import pydyno
print(pydyno.__file__)
from pydyno.discretization.pysb_discretize import PysbDomPath
from pydyno.seqanalysis import SeqAnalysis
from pydyno.visualize_simulations import VisualizeSimulations
from pydyno.visualize_discretization import visualization_path
%matplotlib inline
import random
import h5py
from pydyno.dominant_path_frequency_analysis import FreqAnalysis
import sys 
# sys.path.append('/Users/geenaildefonso/Projects/NERM')
sys.path.append('../')
from model_files.nerm import model as model
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
# from necro_pydream_model_file_8920 import model
from pysb.simulator import ScipyOdeSimulator
import numpy as np
from scipy.stats import norm,uniform
import seaborn as sns
from matplotlib import pyplot as plt
import random
import os
from pylab import *
from pysb.core import *
from pysb.bng import *
from pysb.integrate import *
import matplotlib.pyplot as plt
import numpy as np
from pysb.util import alias_model_components
from scipy import stats
from pysb.simulator import SimulationResult
import pandas as pd
import seaborn as sns

In [ ]:
pars = np.load('../necro_pars10k_allclusmod.npy')

In [ ]:
m = model.components
tspan = np.linspace(0, 1800, 500)

In [ ]:
solverwt0 = ScipyOdeSimulator(model, tspan=tspan, verbose = True)
result10 = solverwt0.run(initials = {m.TNF(brec=None):233}, param_values=pars, num_processors = 20)
# wtclus0.save('h5files5720/pydyno_s36prod_5k_5620_100tnf_cluster0pars_72hrwt.h5',include_obs_exprs = True)
# result10.save('sim_nerm_allpars_tnf10.h5')

In [ ]:
dp_nerm_10 = PysbDomPath(model, result10)

In [ ]:
signs_nerm_10, paths_nerm_10 = dp_nerm_10.get_path_signatures('s36', 'production', depth=10, 
                                                              dom_om=1, num_processors = 20)

In [ ]:
signs_nerm_10.dissimilarity_matrix()

In [ ]:
signs_nerm_10.silhouette_score_spectral_range(range(2,10),  num_processors = 20)

In [ ]:
signs_nerm_10.silhouette_score_agglomerative_range(10 , num_processors = 20) # n_clusters best silhouette score: 

In [ ]:
signs_nerm_10.agglomerative_clustering(2)

In [ ]:
signs_nerm_10.spectral_clustering(2)

In [ ]:
sorted_label_idxs = np.argsort(signs_nerm_10.labels)
sorted_diss = signs_nerm_10.diss[sorted_label_idxs, :]
sorted_diss = sorted_diss[:, sorted_label_idxs]
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
    if not ax:
        ax = plt.gca()
    # Plot the heatmap
    im = ax.imshow(data, **kwargs)
    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")
    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
#     ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)
    return im, cbar
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
im, cbar = heatmap(sorted_diss, range(10000), range(10000), ax=ax,
                   cmap="YlGn", cbarlabel="distance")
ax.figure.axes[-1].yaxis.label.set_size(14)
plt.xlabel('signatures', fontsize = 14)
plt.ylabel('signatures', fontsize = 14)
fig.tight_layout()
# plt.savefig('heatmap_clusters_necro_pydream_5chns_10k_5720_10tnf_aug.pdf')
plt.show()